In [1]:
# coding: utf-8

import tensorflow as tf
import numpy as np

In [2]:
##
## hyper-parameters
##
alpha = 0.1
nn_input_dim = 1
nn_recurrent_layer_neurons = 2
nn_output_neurons = 1
nn_recurrent_deepness = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()

np.random.random((nn_input_dim,nn_recurrent_layer_neurons)) - 1

with tf.name_scope('W_input'):
    W_input = tf.Variable(tf.random_normal( \
        shape=[nn_input_dim, nn_recurrent_layer_neurons], \
        dtype=tf.float32), name='W_input')

with tf.name_scope('b_input'):
    b_input = tf.Variable(1., name='b_input')
    
with tf.name_scope('W_recurrent'):
    W_recurrent = tf.Variable(tf.random_normal( \
        shape=[nn_recurrent_layer_neurons, nn_recurrent_layer_neurons], \
        dtype=tf.float32), name='W_recurrent')

with tf.name_scope('b_recurrent'):
    b_input = tf.Variable(1., name='b_recurrent')

with tf.name_scope('W_output'):    
    W_output = tf.Variable(tf.random_normal(\
        shape=[nn_recurrent_layer_neurons, nn_output_neurons], \
        dtype=tf.float32), name='W_output')

with tf.name_scope('b_output'):
    b_output = tf.Variable(1., name='b_output')

with tf.name_scope('Y_train'):
    Y_train = tf.placeholder(tf.float32, name='Y_train')

In [3]:
## re-current input matrix multiply (inputs: X_train, h_recurrent)
g_recursion = 0

rnn_op = {}
rnn_op_index = 0

X_input = {}
x_input_index = 0

Y_output = {}
y_output_index = 0

def inc_rnn_op_index():
    global rnn_op_index
    rnn_op_index = rnn_op_index +1
    return rnn_op_index - 1

def inc_x_input_index():
    global x_input_index
    x_input_index = x_input_index + 1
    return x_input_index - 1

def inc_y_output_op_index():
    global y_output_index
    y_output_index = y_output_index + 1
    return y_output_index - 1

def inc_g_recursion():
    global g_recursion
    g_recursion = g_recursion + 1
    return g_recursion -1
    
def g_RNN(prev_y_output_recursion_index=0):
    
    global X_input
    global Y_output_op
    global rnn_op
    
    x_input_index_tmp = inc_x_input_index()
    y_output_op_index_tmp = inc_y_output_op_index()
    g_recursion = inc_g_recursion()
    
    # define placeholder for input
    with tf.name_scope('rnn_input' + str(g_recursion)):
           
        X_input[x_input_index_tmp] = tf.placeholder(tf.float32, \
               [None, nn_input_dim], name='X_input' + str(x_input_index_tmp))
        print('[d] placeholder: X_input' + str(x_input_index_tmp), ' created.') 
        
    # define nested operations 
    with tf.name_scope('rnn' + str(g_recursion)):
        
        print('[d] creating operations for node: rnn' + str(g_recursion))
        
        # matmul X * W @t==0
        y_output_a_index = inc_rnn_op_index()
        if prev_y_output_recursion_index == False:
            # bottom of rnn, no linked node, just multiply XW
            rnn_op[y_output_a_index] = tf.matmul(X_input[x_input_index_tmp], \
                  W_input)
        else:
            # add result of previous linked node output, multiply XW + previous Y
            rnn_op[y_output_a_index] = tf.add(tf.matmul(X_input[x_input_index_tmp], \
                  W_input), rnn_op[prev_y_output_recursion_index])
    
        # add bias to computation result
        y_output_b_index = inc_rnn_op_index()
        rnn_op[y_output_b_index] = tf.add(rnn_op[y_output_a_index], b_input)
    
        # activation function is the last ouput operation of rnn node - Y_out
        y_output_c_index = inc_rnn_op_index()
        rnn_op[y_output_c_index] = tf.sigmoid(rnn_op[y_output_b_index], name='Y_out' + str(g_recursion))            
        
        print('[d] Y_output operation for node: rnn' + \
              str(g_recursion) + ' is rnn_op[', str(y_output_c_index), ']')

        # precompute matmul of Y*W_recurrent to reuse with up-nested layer
        y_output_recursion_index = inc_rnn_op_index()
        rnn_op[y_output_recursion_index] = tf.matmul(rnn_op[y_output_c_index], \
              W_recurrent, name='Y_out_rec' + str(g_recursion))
        
        print('[d] Y_out_rec operation for node: rnn' + \
              str(g_recursion) + ' is rnn_op[', str(y_output_recursion_index), ']')

    return y_output_recursion_index

In [4]:
##
## data
##
feed_X_train = np.array([[0.0], [1.0], [1.0], [0.0], [0.0], [0.0], [1.0], [1.0]])
feed_Y_train = np.array([['x'], [1.0], ['x'], [1.0], ['x'], [0.0], ['x'], [0.0]])

In [5]:
# In[ ]:

def test():
    
    print()
    for pointer in range(0, len(feed_X_train), 2):
        pred_res = sess.run([pred], feed_dict={\
                            X_input[0]: feed_X_train[pointer]   [None, :], \
                            X_input[1]: feed_X_train[pointer+1] [None, :]})
    
        print(feed_X_train[pointer], feed_X_train[pointer+1], np.around(pred_res))

    print()
    for pointer in range(0, len(feed_X_train), 2):
        pred_res = sess.run([pred], feed_dict={\
                            X_input[0]: feed_X_train[pointer]   [None, :], \
                            X_input[1]: feed_X_train[pointer+1] [None, :]})

        print(feed_X_train[pointer], feed_X_train[pointer+1], pred_res)

In [6]:
##
## Session
##
with tf.name_scope('computation'):
    # graph definition
    rnn_prediction_chain = g_RNN(g_RNN())
    # prediction from RNN into single neuron
    pred = tf.sigmoid(tf.add(tf.matmul(rnn_op[rnn_prediction_chain], W_output), b_output))
    # cost
    cost_function = tf.reduce_sum(tf.subtract(pred, Y_train) ** 2)
    # optimizer
    learning_rate = 0.25
    optimizer = tf.train.AdamOptimizer().minimize(cost_function)
    #optimizer = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(cost_function)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
    #optimizer = tf.train.MomentumOptimizer(0.2, 0.8).minimize(cost_function)
    
sess.run(tf.global_variables_initializer())
tb_writer = tf.summary.FileWriter("./tensorboard/", sess.graph)

[d] placeholder: X_input0  created.
[d] creating operations for node: rnn0
[d] Y_output operation for node: rnn0 is rnn_op[ 2 ]
[d] Y_out_rec operation for node: rnn0 is rnn_op[ 3 ]
[d] placeholder: X_input1  created.
[d] creating operations for node: rnn1
[d] Y_output operation for node: rnn1 is rnn_op[ 6 ]
[d] Y_out_rec operation for node: rnn1 is rnn_op[ 7 ]


In [7]:
zip_x = {}
zip_y = {}

train_batch_X0 = []
train_batch_X1 = []
train_batch_Y = []

# print logical function    
for pointer in range(0, len(feed_X_train), 2):
    print(feed_X_train[pointer], feed_X_train[pointer+1], \
          ' Xor ', feed_Y_train[pointer+1])
    train_batch_X0.append(feed_X_train[pointer])
    train_batch_X1.append(feed_X_train[pointer+1])
    train_batch_Y.append(feed_Y_train[pointer+1])
    
    
print('[i] train batches: ')
print('X0: ', train_batch_X0)
print('X1: ', train_batch_X1)
print('Y: ',  train_batch_Y)

epoch = -1
cost_function_res = 1000

print('[i] training')
while cost_function_res > 0.001:
#for i in range(10000):
    epoch = epoch + 1
    for pointer in range(0, len(feed_X_train), 2):
        
        optimizer_res, cost_function_res = sess.run([optimizer, cost_function], \
            feed_dict={X_input[0]: [[0], [0], [1], [1]], \
                       X_input[1]: [[0], [1], [0], [1]], \
                       Y_train:    [[0], [1], [1], [0]]})

    if epoch % 1000 == 0:
        print('epoch: ', epoch, ' cost: ', cost_function_res)
        
    if epoch > 100000:
        break

print('[i] training finished, cost: ', cost_function_res)

[ 0.] [ 1.]  Xor  ['1.0']
[ 1.] [ 0.]  Xor  ['1.0']
[ 0.] [ 0.]  Xor  ['0.0']
[ 1.] [ 1.]  Xor  ['0.0']
[i] train batches: 
X0:  [array([ 0.]), array([ 1.]), array([ 0.]), array([ 1.])]
X1:  [array([ 1.]), array([ 0.]), array([ 0.]), array([ 1.])]
Y:  [array(['1.0'], 
      dtype='<U3'), array(['1.0'], 
      dtype='<U3'), array(['0.0'], 
      dtype='<U3'), array(['0.0'], 
      dtype='<U3')]
[i] training
epoch:  0  cost:  1.84884
epoch:  1000  cost:  0.245729
epoch:  2000  cost:  0.00474982
[i] training finished, cost:  0.000998753
